In [1]:
from models.lag_ml_model import scaling_function,preprocess_data,classifier_models,train_model,grid_optimize_model,random_optimize_model,get_param_grids,get_data

In [2]:
df = get_data(path=None)

In [3]:
X_train,X_test,y_train,y_test = preprocess_data(df)

In [4]:
X_train

,Close,High,Low,Open,Volume,sentiment_score,Close_lag1,Close_lag2,Close_lag3,Close_lag4,...,Close_lag51,Close_lag52,Close_lag53,Close_lag54,Close_lag55,Close_lag56,Close_lag57,Close_lag58,Close_lag59,Close_lag60
Date,,,,,,,,,,,,,,,,,,,,,
2015-03-31,31.107500,31.622499,31.090000,31.522499,168362400,0.080131,31.592501,30.812500,31.059999,30.844999,...,26.705000,27.450001,27.555000,27.312500,28.002501,27.972500,26.937500,26.565001,26.562500,27.332500
2015-04-01,31.062500,31.280001,30.775000,31.205000,162485600,0.540406,31.107500,31.592501,30.812500,31.059999,...,26.497499,26.705000,27.450001,27.555000,27.312500,28.002501,27.972500,26.937500,26.565001,26.562500
2015-04-02,31.330000,31.389999,31.047501,31.257500,128880400,0.342493,31.062500,31.107500,31.592501,30.812500,...,27.180000,26.497499,26.705000,27.450001,27.555000,27.312500,28.002501,27.972500,26.937500,26.565001
2015-04-06,31.837500,31.877501,31.082500,31.117500,148776000,0.887742,31.330000,31.062500,31.107500,31.592501,...,27.387501,27.180000,26.497499,26.705000,27.450001,27.555000,27.312500,28.002501,27.972500,26.937500
2015-04-07,31.502501,32.029999,31.495001,31.910000,140049200,0.859289,31.837500,31.330000,31.062500,31.107500,...,28.100000,27.387501,27.180000,26.497499,26.705000,27.450001,27.555000,27.312500,28.002501,27.972500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-25,164.850006,165.119995,160.869995,163.839996,91974200,0.887742,162.740005,160.070007,164.320007,167.300003,...,175.740005,179.449997,174.559998,175.080002,171.179993,165.320007,161.839996,163.759995,164.770004,165.300003
2022-02-28,165.119995,165.419998,162.429993,163.059998,95056600,0.018562,164.850006,162.740005,160.070007,164.320007,...,174.330002,175.740005,179.449997,174.559998,175.080002,171.179993,165.320007,161.839996,163.759995,164.770004
2022-03-01,163.199997,166.600006,161.970001,164.699997,83474400,0.887742,165.119995,164.850006,162.740005,160.070007,...,179.300003,174.330002,175.740005,179.449997,174.559998,175.080002,171.179993,165.320007,161.839996,163.759995


In [5]:
X_test

,Close,High,Low,Open,Volume,sentiment_score,Close_lag1,Close_lag2,Close_lag3,Close_lag4,...,Close_lag51,Close_lag52,Close_lag53,Close_lag54,Close_lag55,Close_lag56,Close_lag57,Close_lag58,Close_lag59,Close_lag60
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-04,163.169998,165.550003,162.100006,164.490005,83737200,0.817217,166.229996,166.559998,163.199997,165.119995,...,169.750000,171.139999,172.259995,179.300003,174.330002,175.740005,179.449997,174.559998,175.080002,171.179993
2022-03-07,159.300003,165.020004,159.039993,163.360001,96418800,0.065044,163.169998,166.229996,166.559998,163.199997,...,172.990005,169.750000,171.139999,172.259995,179.300003,174.330002,175.740005,179.449997,174.559998,175.080002
2022-03-08,157.440002,162.880005,155.800003,158.820007,131148300,0.858770,159.300003,163.169998,166.229996,166.559998,...,175.639999,172.990005,169.750000,171.139999,172.259995,179.300003,174.330002,175.740005,179.449997,174.559998
2022-03-09,162.949997,163.410004,159.410004,161.479996,91454900,-0.593222,157.440002,159.300003,163.169998,166.229996,...,176.279999,175.639999,172.990005,169.750000,171.139999,172.259995,179.300003,174.330002,175.740005,179.449997
2022-03-10,158.520004,160.389999,155.979996,160.199997,105342000,-0.261068,162.949997,157.440002,159.300003,163.169998,...,180.330002,176.279999,175.639999,172.990005,169.750000,171.139999,172.259995,179.300003,174.330002,175.740005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-21,245.550003,248.690002,245.220001,245.949997,53197400,0.009650,245.830002,244.869995,244.470001,244.600006,...,243.039993,243.009995,242.649994,239.589996,237.330002,234.929993,235.059998,232.869995,229.869995,228.520004
2025-02-24,247.100006,248.860001,244.419998,244.929993,51326400,0.700623,245.550003,245.830002,244.869995,244.470001,...,242.839996,243.039993,243.009995,242.649994,239.589996,237.330002,234.929993,235.059998,232.869995,229.869995
2025-02-25,247.039993,250.000000,244.910004,248.000000,48013300,0.833926,247.100006,245.550003,245.830002,244.869995,...,246.750000,242.839996,243.039993,243.009995,242.649994,239.589996,237.330002,234.929993,235.059998,232.869995


In [6]:
X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)
print(X_train_scaled.shape,X_test_scaled.shape,y_train.shape,y_test.shape)
classifier = classifier_models()
results,y_pred_dict = train_model(classifiers=classifier, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
best_base = results.loc[results['Accuracy'].idxmax(), 'Model']


(1745, 66) (749, 66) (1745,) (749,)

Logistic Regression Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest Performance:
Accuracy: 0.6008
Confusion Matrix:
[[291  64]
 [235 159]]
              precision    recall  f1-score   support

           0       0.55      0.82      0.66       355
           1       0.71      0.40      0.52       394

    accuracy                           0.60       749
   macro avg       0.63      0.61      0.59       749
weighted avg       0.64      0.60      0.58       749


XGBoost Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f1-score   suppor

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.5981
Confusion Matrix:
[[260  95]
 [206 188]]
              precision    recall  f1-score   support

           0       0.56      0.73      0.63       355
           1       0.66      0.48      0.56       394

    accuracy                           0.60       749
   macro avg       0.61      0.60      0.59       749
weighted avg       0.61      0.60      0.59       749


K-Nearest Neighbors Performance:
Accuracy: 0.6649
Confusion Matrix:
[[217 138]
 [113 281]]
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       355
           1       0.67      0.71      0.69       394

    accuracy                           0.66       749
   macro avg       0.66      0.66      0.66       749
weighted avg       0.66      0.66      0.66       749


Decision Tree Performance:
Accuracy: 0.5794
Confusion Matrix:
[[ 59 296]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.76

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [10]:
y_pred_dict

{'Logistic Regression': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1

In [11]:
from backtesting import Backtest,Strategy
def backtest(y_pred, X_test):
    def get_signal():
        return y_pred
    class MyStrategy(Strategy):
        def init(self):
            self.signal = self.I(get_signal)
        def next(self):
            if self.signal == 1:
                if not self.position:
                    self.buy()
            elif self.signal == 0:
                if self.position:
                    self.position.close()

    bt = Backtest(X_test,MyStrategy,cash=10000,commission=0.0002)
    stats = bt.run()
    print(stats)
    return stats

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)

Backtest.run:   0%|          | 0/748 [00:00<?, ?it/s]

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    55.00668
Equity Final [$]                     9894.181
Equity Peak [$]                   12249.29839
Commissions [$]                      87.26928
Return [%]                           -1.05819
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    -0.35728
Volatility (Ann.) [%]                17.30052
CAGR [%]                             -0.24542
Sharpe Ratio                         -0.02065
Sortino Ratio                         -0.0284
Calmar Ratio                         -0.01659
Alpha [%]                           -18.72363
Beta                                  0.38884
Max. Drawdown [%]                   -21.53146
Avg. Drawdown [%]                     -6.7302
Max. Drawdown Duration      344 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    31.37517
Equity Final [$]                  10816.93508
Equity Peak [$]                   11051.28608
Commissions [$]                      45.02439
Return [%]                            8.16935
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     2.67727
Volatility (Ann.) [%]                 14.5793
CAGR [%]                              1.83039
Sharpe Ratio                          0.18364
Sortino Ratio                         0.27148
Calmar Ratio                          0.10211
Alpha [%]                            -4.32209
Beta                                  0.27495
Max. Drawdown [%]                   -26.21955
Avg. Drawdown [%]                     -6.9088
Max. Drawdown Duration      608 days 00:00:00
Avg. Drawdown Duration      151 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    27.63685
Equity Final [$]                  14093.88819
Equity Peak [$]                   15375.36737
Commissions [$]                       9.94178
Return [%]                           40.93888
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.23833
Volatility (Ann.) [%]                30.08641
CAGR [%]                              8.24884
Sharpe Ratio                          0.40677
Sortino Ratio                         0.67242
Calmar Ratio                           0.4172
Alpha [%]                            -2.71274
Beta                                  0.96083
Max. Drawdown [%]                   -29.33467
Avg. Drawdown [%]                    -6.80479
Max. Drawdown Duration      456 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    52.33645
Equity Final [$]                  11979.26774
Equity Peak [$]                   13701.26407
Commissions [$]                      90.70783
Return [%]                           19.79268
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     6.26439
Volatility (Ann.) [%]                16.96957
CAGR [%]                              4.25956
Sharpe Ratio                          0.36915
Sortino Ratio                         0.57775
Calmar Ratio                          0.30236
Alpha [%]                             3.91445
Beta                                   0.3495
Max. Drawdown [%]                   -20.71803
Avg. Drawdown [%]                    -5.18858
Max. Drawdown Duration      316 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

...

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    25.36716
Equity Final [$]                  10445.58083
Equity Peak [$]                   11401.26022
Commissions [$]                       59.4019
Return [%]                            4.45581
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     1.47752
Volatility (Ann.) [%]                25.14159
CAGR [%]                              1.01202
Sharpe Ratio                          0.05877
Sortino Ratio                         0.08552
Calmar Ratio                          0.03529
Alpha [%]                           -32.52845
Beta                                  0.81407
Max. Drawdown [%]                   -41.86406
Avg. Drawdown [%]                   -16.05859
Max. Drawdown Duration     1001 days 00:00:00
Avg. Drawdown Duration      269 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    22.02937
Equity Final [$]                  14256.70141
Equity Peak [$]                   15559.90057
Commissions [$]                      11.09727
Return [%]                           42.56701
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.67291
Volatility (Ann.) [%]                30.71886
CAGR [%]                              8.53641
Sharpe Ratio                          0.41254
Sortino Ratio                         0.68373
Calmar Ratio                          0.38244
Alpha [%]                            -2.21697
Beta                                  0.98575
Max. Drawdown [%]                   -33.13689
Avg. Drawdown [%]                    -7.05506
Max. Drawdown Duration      455 days 00:00:00
Avg. Drawdown Duration       61 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    35.78104
Equity Final [$]                  13965.22568
Equity Peak [$]                   16257.12196
Commissions [$]                      68.02299
Return [%]                           39.65226
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    11.89255
Volatility (Ann.) [%]                14.22285
CAGR [%]                              8.01978
Sharpe Ratio                          0.83616
Sortino Ratio                          1.5326
Calmar Ratio                          0.84358
Alpha [%]                            29.63891
Beta                                  0.22041
Max. Drawdown [%]                    -14.0978
Avg. Drawdown [%]                    -2.66208
Max. Drawdown Duration      226 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    64.21896
Equity Final [$]                  10143.75017
Equity Peak [$]                   11928.38363
Commissions [$]                      65.17347
Return [%]                             1.4375
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     0.48136
Volatility (Ann.) [%]                18.35461
CAGR [%]                              0.33022
Sharpe Ratio                          0.02623
Sortino Ratio                         0.03739
Calmar Ratio                          0.01936
Alpha [%]                           -18.46797
Beta                                  0.43815
Max. Drawdown [%]                   -24.85895
Avg. Drawdown [%]                    -7.20349
Max. Drawdown Duration      514 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    61.68224
Equity Final [$]                  14097.94656
Equity Peak [$]                   15379.42573
Commissions [$]                      77.40136
Return [%]                           40.97947
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.24921
Volatility (Ann.) [%]                26.03006
CAGR [%]                              8.25604
Sharpe Ratio                          0.47058
Sortino Ratio                         0.77892
Calmar Ratio                          0.38187
Alpha [%]                             7.47389
Beta                                   0.7375
Max. Drawdown [%]                   -32.07703
Avg. Drawdown [%]                     -6.9128
Max. Drawdown Duration      664 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

In [18]:
best_model = classifier[best_base]

In [20]:
y_pred = best_model.predict(X_test_scaled)

In [33]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749

0.8878504672897196
[[310  45]
 [ 39 355]]


In [23]:
!pip install backtesting

In [26]:
get_signal()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
from backtesting import Backtest,Strategy
def get_signal():
    return y_test
class MyStrategy(Strategy):
    def init(self):
        self.signal = self.I(get_signal)
    def next(self):
        if self.signal == 1:
            if not self.position:
                self.buy()
        elif self.signal == 0:
            if self.position:
                self.position.close()

bt = Backtest(X_test,MyStrategy,cash=10000,commission=0.0002)
stats = bt.run()
stats

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    51.26836
Equity Final [$]                  17462.47122
Equity Peak [$]                   18473.92066
Commissions [$]                      69.04271
Return [%]                           74.62471
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    20.63022
Volatility (Ann.) [%]                20.40953
CAGR [%]                             13.74223
Sharpe Ratio                          1.01081
Sortino Ratio                         1.94515
Calmar Ratio                          1.54451
Alpha [%]                             57.2394
Beta                                  0.38267
Max. Drawdown [%]                   -13.35712
Avg. Drawdown [%]                    -2.91981
Max. Drawdown Duration      226 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    55.00668
Equity Final [$]                     9894.181
Equity Peak [$]                   12249.29839
Commissions [$]                      87.26928
Return [%]                           -1.05819
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    -0.35728
Volatility (Ann.) [%]                17.30052
CAGR [%]                             -0.24542
Sharpe Ratio                         -0.02065
Sortino Ratio                         -0.0284
Calmar Ratio                         -0.01659
Alpha [%]                           -18.72363
Beta                                  0.38884
Max. Drawdown [%]                   -21.53146
Avg. Drawdown [%]                     -6.7302
Max. Drawdown Duration      344 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                          

In [30]:
stats

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    51.26836
Equity Final [$]                  17462.47122
Equity Peak [$]                   18473.92066
Commissions [$]                      69.04271
Return [%]                           74.62471
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    20.63022
Volatility (Ann.) [%]                20.40953
CAGR [%]                             13.74223
Sharpe Ratio                          1.01081
Sortino Ratio                         1.94515
Calmar Ratio                          1.54451
Alpha [%]                             57.2394
Beta                                  0.38267
Max. Drawdown [%]                   -13.35712
Avg. Drawdown [%]                    -2.91981
Max. Drawdown Duration      226 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                          